Hello, this Machine learning project is to model the credit card data, and predict whether a transaction is fraudulent or not

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

IMPORT DATA

In [ ]:
ccd = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
ccd.head(10)


In [ ]:
ccd.describe()

As part of pre-processing the data, I will scale the amount column down since it has outliers that vary too much  

In [ ]:
from sklearn.preprocessing import RobustScaler
#ccd['Amount']=ccd['Amount'].to_numpy().reshape(-1,1)
ccd['Amount']=RobustScaler().fit_transform(ccd['Amount'].to_numpy().reshape(-1,1))

In [ ]:
# standard scaler was not working
ccd['Time'] = (ccd['Time']-ccd['Time'].min())/ (ccd['Time'].max()- ccd['Time'].min())

In [ ]:
ccd.describe()

Time and Amount columns scaled down successfuly,
Now lets split the data Independant and dependant variables also known as features and targets

In [ ]:
x = ccd.drop(columns = 'Class', axis = 1)
y = ccd['Class']
print(x.shape, y.shape)


Now lets split the data into training, testing sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size = 0.2, stratify = y, random_state = 4)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)

Lets train the logistic regression model

In [ ]:
#LogisticRegression().fit(x_train,y_train)


Ran into the error: faliure to converge,
Solution: undersample the dataset
This is mainly due to the imbalanced nature of the dataset, so we now have to balance the dataset, we do that by matching the number of fraudulent classes with the number of non-fraudulent classes 

Undersampling

In [ ]:
# number of fraudulent transactions in the data 
print(ccd['Class'].value_counts())

As we can see, there are 492 fraudulent transactions,
Now lets balance the dataset,

In [ ]:
non_fraud = ccd[ccd.Class == 0]
fraud = ccd[ccd.Class == 1]

In [ ]:
df = non_fraud.sample(492)
df = pd.concat([df,fraud],axis = 0)

df is now our balanced dataset 

In [ ]:
df.shape

Now lets repeat our previous steps,

In [ ]:
x = df.drop(columns = 'Class', axis = 1)
y = df['Class']
print(x.shape, y.shape)


In [ ]:
x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size = 0.2, stratify = y, random_state = 4)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)

In [ ]:
model = LogisticRegression()
model.fit(x_train,y_train)

We have succesfully trained the model, Now lets check how good it is...

In [ ]:
prediction = model.predict(x_test)


In [ ]:
print(classification_report(y_test, prediction))
print(accuracy_score(y_test, prediction))

There we have it, we have 92 % accurate model with really good precision,recall, f1 score and support reports. 